In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats.stats import pearsonr
from feather import read_dataframe
%matplotlib inline
from functools import lru_cache
import time
from tqdm import tqdm_notebook
import logging

In [2]:
complete = read_dataframe("user_book_ratings.feather",).pivot_table("Book-Rating",
                                                                    index="User-ID",
                                                                    columns="Book-Title")

In [3]:
complete.head(10)

Book-Title    Murder of a Sleeping Beauty (Scumble River Mysteries (Paperback))  \
User-ID                                                                           
User_100004                                                NaN                    
User_100009                                                NaN                    
User_10001                                                 NaN                    
User_100010                                                NaN                    
User_100038                                                NaN                    
User_100053                                                NaN                    
User_100066                                                NaN                    
User_100088                                                NaN                    
User_100098                                                NaN                    
User_100115                                                NaN                    

Book-Title    Q-Space (Star Trek The Next Generation, Book 47)  \
User-ID                                                          
User_100004                                                NaN   
User_100009                                                NaN   
User_10001                                                 NaN   
User_100010                                                NaN   
User_100038                                                NaN   
User_100053                                                NaN   
User_100066                                                NaN   
User_100088                                                NaN   
User_100098                                                NaN   
User_100115                                                NaN   

Book-Title    Q-Zone (Star Trek The Next Generation, Book 48)  \
User-ID                                                         
User_100004                                               NaN   
User_100009                                               NaN   
User_10001                                                NaN   
User_100010                                               NaN   
User_100038                                               NaN   
User_100053                                               NaN   
User_100066                                               NaN   
User_100088                                               NaN   
User_100098                                               NaN   
User_100115                                               NaN   

Book-Title   " Lamb to the Slaughter and Other Stories (Penguin 60s S.)  \
User-ID                                                                   
User_100004                                                NaN            
User_100009                                                NaN            
User_10001                                                 NaN            
User_100010                                                NaN            
User_100038                                                NaN            
User_100053                                                NaN            
User_100066                                                NaN            
User_100088                                                NaN            
User_100098                                                NaN            
User_100115                                                NaN            

Book-Title   "A" is for Alibi : A Kinsey Millhone Mystery (A Kinsey Millhone Mystery)  \
User-ID                                                                                 
User_100004                                                NaN                          
User_100009                                                NaN                          
User_10001                                                 NaN                          
User_100010                                                NaN                     

In [4]:
complete.memory_usage().sum() / 1024 ** 3 # GB

1.2238024920225143

In [5]:
############# Utilities #############


def compute_evaluation_metric(y_true, y_pred, metric="MSE"):
    """
        Given two vectors computes the selected
        metric over non-missing values.
    """
    
    # This works only on the common elements
    # missing values are ignored
    diff = np.subtract(y_true, y_pred)
    
    # Count NaNs in the difference, check for NaNs and 
    # invert that, finally count non-zeros 
    # https://stackoverflow.com/a/21778195/6655150
    N = np.count_nonzero(~np.isnan(diff))
    
    if metric == "MSE":
        distance = (1 / N) * np.nansum( np.power(diff, 2) )
    elif metric == "MAE":
        distance = (1 / N) * np.nansum( np.abs(diff) )
    elif metric == "RMSE":
        distance = np.sqrt((1 / N) * np.nansum( np.power(diff, 2) ))
    elif metric == "R2":
        y_mean = np.nanmean(y_true)
        diffs1 = np.nansum(np.square(y_true - y_mean))
        diffs2 = np.nansum(np.square(y_pred - y_true))
        distance = 1 - diffs2 / diffs1
        
    return distance


def cos_sim(user, others):
    """
        This is nothing more than the implementation of cosine similarity
        in a way to handle (ignore) missing values. It is about 2-3x faster
        than replacing NaNs with zeros and then using scipy's or sklearn's
        functions. It assumes the input vectors are `atleast_2D`.
    """
    
    # add some numbers for numeric stability
    sum_sq = np.nansum(np.multiply(user, others), axis=1)

    magnitudes = np.multiply(np.sqrt(np.nansum(np.square(user))),
                             np.sqrt(np.nansum(np.square(others), axis=1))) 
    
    # calculate cosing similarity and add 
    # a small amount to avoid division by 0
    return np.divide(sum_sq+1e-7, magnitudes+1e-3).reshape((user.shape[0], -1))

def pearson_sim(user, others):
    """
        Similarly to the calculation of cosine similarity, this custom approach
        is faster (~1.5x) than filling the whole matrix (this time with the row
        mean) and then using `scipy.stats.stats.pearsonr` with a list comprehension
        iterating over all the rows.
    """
    
    user_mean = np.nanmean(user)
    others_means = np.nanmean(others, axis=1, keepdims=True)

    diff1 = np.subtract( user, user_mean ) 
    diff2 = np.subtract( others, others_means )

    var1 = np.sqrt( np.nansum( np.power(diff1, 2) ) )
    var2 = np.sqrt( np.nansum( np.power(diff2, 2), axis=1, 
                               keepdims=True) )

    num = np.nansum( np.multiply( diff1, diff2 ), axis=1,
                     keepdims=True )
    denom = np.multiply( var1, var2 )
    
    return np.divide(num, (denom + 1e-3)).reshape((user.shape[0], -1))


def similirity_evaluation(user, others, method="pearson"):
    """
        Given two vectors, or a vector and list of vectors, computes
        the selected similarity metric(s). Make sure both vectors
        are 2D.
    """

    if method == "pearson":
        return pearson_sim(user, others)
    
    elif method == "cosine":
        return cos_sim(user, others)

# mycls . lazypropINST . __get__ myclsFUNC

class lazyproperty:
    
    def __init__(self, func):
        self.func = func
        
    def __get__(self, instance, cls):
#         if instance is None:
#             print("First run")
#             return self
#         else:
        value = self.func(instance)
        setattr(instance, self.func.__name__, value)
        return value
    
############# Functions used for actual prediction #############

class BaseRecommender:
    """
        This showcases the general pattern: subclasses need only
        implement _get_item_sims and _get_user_sims which calculate
        item and user similarities for a given input string
    """
    
    def __init__(self, df, *args, **kwargs):
        self.df = df
        
        # so we know where the missing values are
        self.nan_mask = df.isna() 
        
    def _get_user(self, i):
        """Get row i; should be the same dtype as RowIndex"""
        return np.atleast_2d(self.df.loc[i].values)

    def _get_item(self, j):
        """Get row j from the transponsed matrix, aka item j"""
        return np.atleast_2d(self.df.loc[:,j].values)
    
    # TODO: Better function signatures and more details on
    # how to overwrite them        
    def _get_item_sims(self, *args, **kwargs):
        raise NotImplementedError
    
    def _get_user_sims(self, *args, **kwargs):
        raise NotImplementedError
    
    def find_similar_items(self, item_name=None, metric="cosine"):
        """
            item_name: str or list of str, columns of dataframe
            metric: str, one of pearson / cosine
        
            Given an item and a dataframe (optionally a similarity metric)
            this function finds the most similar items.
        """
        
        if item_name is not None:
            sims = self._get_item_sims(item_name, metric)
        else:
            raise Exception("Invalid user or vector input.")
            
        results = pd.DataFrame([self.df.columns, sims.ravel()],
                               index=["Items", "Similarity scores"]).T
        
        return results.sort_values("Similarity scores", ascending=False).reset_index(drop=True)
    
    
    def find_similar_users(self, username=None, metric="pearson"):
        """
            username: str, index of dataframe
            metric: str, one of pearson / cosine
        
            Given a username (optionally a similarity metric) or a vector
            this function finds the most similar users.
        """
        
        if username is not None:
            sims = self._get_user_sims(username, metric)
        else:
            raise Exception("Invalid user input.")

        results = pd.DataFrame([self.df.index, sims.ravel()],
                               index=["Users", "Similarity scores"]).T
        
        return results.sort_values("Similarity scores", ascending=False).reset_index(drop=True)
    
class LowMemRecommender(BaseRecommender):
    """
        Does not fill missing values, does not precompute anything,
        and does not store excessive arrays. Inference is usually
        slower but calculations are more accurate due to better
        handling of NaNs (does not assume NaNs to be zero).
    """


    def _get_user_sims(self, username, metric="cosine"):
        user = self._get_user(username) # get vectors
        return similirity_evaluation(user, self.df, metric) # calculate similarity
    
    def _get_item_sims(self, item_name, metric="cosine"):
        item = self._get_item(item_name) # get vectors
        return similirity_evaluation(item, self.df.T, metric) # calculate similarity
    
    
    ## THIS MAY NOT BE WORKING CORRECTLY FOR ITEMS
    ## TODO: Move to base class
    def predict_user_ratings(self, username=None, metric="cosine", *, vector=None):
        """
            username: str, index of dataframe
            metric: str, one of pearson / cosine
            vector: vector with shape (n_items,) or (1,n_items), ignored
                    if username is also specified
        
            Given a certain username or a vector, it tries
            to predict missing values for each user/row by taking into account
            the user(s) mean and their similarity with other users.
        """
        
        if username is not None:
            user = self._get_user(username)
        elif vector is not None:
            # make sure the shape is correct
            user = np.atleast_2d(vector)
        else:
            raise Exception("Invalid user or vector input.")

        # use numpy to calculate the mean ignoring NaNs
        user_mean = np.nanmean(user)
        others_means =  np.nanmean(self.df, axis=1, keepdims=True)

        diff2 = np.subtract(self.df, others_means)

        similarities = similirity_evaluation(user, self.df, metric)

        num = np.nansum( np.multiply(similarities.T, diff2), axis=0 )
        denom = np.nansum(similarities)

        return np.add(user_mean, np.divide(num, denom + 1e-4))
    
    
class FastRecommender(BaseRecommender):
    """
        This implementation stores a copy of the original (unless copy=False)
        as well as a mask of missing values
    """
    
    def __init__(self, df, copy=True):
        
        super().__init__(df)
        
        if copy:
            # in case we need to reset
            self._orig_df = df.copy() 
        
    @lazyproperty
    def user_similarities(self):
        logging.info("Computing user similarity matrix...")
        return np.corrcoef(complete.fillna(0).values + 1e-3) # numerical stability
    
    @lazyproperty
    def item_similarities(self):
        logging.info("Computing item similarity matrix...")
        return np.corrcoef(complete.T.fillna(0).values + 1e-3) # numerical stability
    
    def _get_user_index(self, i):
        """Get row i; should be the same dtype as RowIndex"""
        return complete.index.get_loc(i)

    def _get_item_index(self, j):
        """Get row j from the transponsed matrix, aka item j"""
        return complete.columns.get_loc(j)
    
    def _get_item_sims(self, item, *args, **kwargs):
        idx = self._get_item_index(item)
        return self.item_similarities[idx]
    
    def _get_user_sims(self, user, *args, **kwargs):
        idx = self._get_user_index(user)
        return self.user_similarities[idx]
    
    
# Cheating
def Recommender(df, low_memory=False):
    
    if low_memory:
        return LowMemRecommender(df)
    else:
        return FastRecommender(df)
        

In [28]:
start = time.time()
rec = Recommender(complete) # instant fetch, slow start (~30 sec)
end = time.time()

end - start

1.5803501605987549

In [29]:
rec.find_similar_users("User_8", "cosine")

Users Similarity scores
0           User_8                 1
1       User_67544          0.398428
2      User_116866          0.311722
3      User_219008          0.275284
4      User_263325          0.134649
5       User_11676         0.0180414
6       User_16759       2.82467e-15
7      User_129002       2.82467e-15
8       User_87846       2.82467e-15
9      User_220626       2.82467e-15
10     User_171795       2.82467e-15
11     User_199627       2.82467e-15
12      User_70688       2.82467e-15
13     User_129110       2.82467e-15
14     User_220699       2.82467e-15
15      User_26831       2.82467e-15
16     User_128949       2.82467e-15
17     User_220620       2.82467e-15
18      User_70648       2.82467e-15
19     User_171959       2.82467e-15
20     User_268392       2.82467e-15
21     User_268416       2.82467e-15
22     User_128574       2.82467e-15
23      User_12854       2.82467e-15
24     User_172023       2.82467e-15
25     User_268589       2.82467e-15
26      User_52025       2.82467e-15
27      User_87758       2.82467e-15
28     User_171604       2.82467e-15
29     User_171703       2.82467e-15
...            ...               ...
12460  User_112001      -0.000974409
12461  User_265889      -0.000975171
12462  User_107784       -0.00098974
12463  User_177432      -0.000992086
12464  User_189835      -0.000997208
12465  User_100906       -0.00100425
12466  User_248718       -0.00100893
12467   User_13552       -0.00101262
12468  User_234828       -0.00101366
12469   User_89602        -0.0010437
12470  User_162639       -0.00104407
12471   User_31315       -0.00105224
12472   User_78973       -0.00106587
12473   User_23902       -0.00106911
12474    User_7346       -0.00108598
12475  User_171118       -0.00110346
12476  User_257204       -0.00114171
12477   User_60244       -0.00120428
12478   User_35859       -0.00125806
12479  User_135149       -0.00125951
12480  User_204864       -0.00126407
12481  User_235105       -0.00127388
12482  User_158295       -0.00131803
12483  User_123883       -0.00135056
12484  User_104636       -0.00137033
12485   User_95359       -0.00148947
12486  User_114368       -0.00149221
12487   User_16795       -0.00161383
12488  User_153662       -0.00182692
12489   User_98391       -0.00247812

[12490 rows x 2 columns]

In [30]:
rec.find_similar_users("User_8", "cosine")

Users Similarity scores
0           User_8                 1
1       User_67544          0.398428
2      User_116866          0.311722
3      User_219008          0.275284
4      User_263325          0.134649
5       User_11676         0.0180414
6       User_16759       2.82467e-15
7      User_129002       2.82467e-15
8       User_87846       2.82467e-15
9      User_220626       2.82467e-15
10     User_171795       2.82467e-15
11     User_199627       2.82467e-15
12      User_70688       2.82467e-15
13     User_129110       2.82467e-15
14     User_220699       2.82467e-15
15      User_26831       2.82467e-15
16     User_128949       2.82467e-15
17     User_220620       2.82467e-15
18      User_70648       2.82467e-15
19     User_171959       2.82467e-15
20     User_268392       2.82467e-15
21     User_268416       2.82467e-15
22     User_128574       2.82467e-15
23      User_12854       2.82467e-15
24     User_172023       2.82467e-15
25     User_268589       2.82467e-15
26      User_52025       2.82467e-15
27      User_87758       2.82467e-15
28     User_171604       2.82467e-15
29     User_171703       2.82467e-15
...            ...               ...
12460  User_112001      -0.000974409
12461  User_265889      -0.000975171
12462  User_107784       -0.00098974
12463  User_177432      -0.000992086
12464  User_189835      -0.000997208
12465  User_100906       -0.00100425
12466  User_248718       -0.00100893
12467   User_13552       -0.00101262
12468  User_234828       -0.00101366
12469   User_89602        -0.0010437
12470  User_162639       -0.00104407
12471   User_31315       -0.00105224
12472   User_78973       -0.00106587
12473   User_23902       -0.00106911
12474    User_7346       -0.00108598
12475  User_171118       -0.00110346
12476  User_257204       -0.00114171
12477   User_60244       -0.00120428
12478   User_35859       -0.00125806
12479  User_135149       -0.00125951
12480  User_204864       -0.00126407
12481  User_235105       -0.00127388
12482  User_158295       -0.00131803
12483  User_123883       -0.00135056
12484  User_104636       -0.00137033
12485   User_95359       -0.00148947
12486  User_114368       -0.00149221
12487   User_16795       -0.00161383
12488  User_153662       -0.00182692
12489   User_98391       -0.00247812

[12490 rows x 2 columns]

In [63]:
rec.find_similar_items("Harry Potter and the Chamber of Secrets (Book 2)")

Items Similarity scores
0       Harry Potter and the Chamber of Secrets (Book 2)                 1
1      Harry Potter and the Prisoner of Azkaban (Book 3)           0.57121
2           Harry Potter and the Goblet of Fire (Book 4)          0.562486
3         Harry Potter and the Sorcerer's Stone (Book 1)          0.443944
4      Harry Potter and the Order of the Phoenix (Boo...          0.350768
5      Harry Potter and the Sorcerer's Stone (Harry P...          0.334566
6      The Fellowship of the Ring (The Lord of the Ri...          0.148811
7                    The Hobbit: or There and Back Again          0.130225
8      Dr. Seuss's A B C (I Can Read It All by Myself...          0.117636
9            Ramona the Pest (Ramona Quimby (Paperback))          0.115795
10                The Canterbury Tales (Bantam Classics)          0.114858
11                                          Lover Beware          0.114778
12                      Charlotte's Web (Trophy Newbery)          0.109757
13     The Reptile Room (A Series of Unfortunate Even...          0.108187
14                                       A Dash of Death          0.104931
15                                       The Giving Tree          0.104796
16     Bard: The Odyssey of the Irish (Celtic World o...          0.103392
17     Dragons of a Fallen Sun (Dragonlance: The War ...          0.103342
18     The Bad Beginning (A Series of Unfortunate Eve...          0.102525
19       Test of the Twins (Dragonlance Legends, Vol. 3)          0.101435
20            Kissed by Shadows (Get Connected Romances)          0.100831
21         J. K. Rowling: The Wizard Behind Harry Potter          0.100258
22     Charlie and the Great Glass Elevator (Puffin N...          0.100113
23                                      The Silmarillion         0.0991179
24     The Secret of the Indian (Indian in the Cupboard)         0.0985802
25                                         The Outsiders         0.0983773
26     The Wakefields of Sweet Valley (Sweet Valley S...          0.097302
27                       Candide (Dover Thrift Editions)         0.0963294
28        The Two Towers (The Lord of the Rings, Part 2)         0.0960474
29                                 Tears Of The Renegade         0.0956369
...                                                  ...               ...
13120                    Behind the Scenes at the Museum       -0.00635491
13121                                         Eden Close       -0.00639625
13122                                      Reap the Wind       -0.00640521
13123     Stalker: A Peter Decker and Rina Lazarus Novel       -0.00646043
13124                                         Snow Crash       -0.00648118
13125  The Green Mile: Night Journey (Green Mile Series)       -0.00651767
13126         Full Blast (Janet Evanovich's Full Series)       -0.00655862
13127                                      Guilty as Sin       -0.00661333
13128                      Women Who Run with the Wolves        -0.0066273
13129                   Revenge of the Middle-Aged Woman       -0.00664093
13130                                       Grand Avenue       -0.00669512
13131                                   The Last Suppers       -0.00672342
13132                    Dr. Atkins' New Diet Revolution        -0.0067242
13133                                          Prime Cut       -0.00672754
13134                                       Dragon Tears       -0.00680591
13135   A Virtuous Woman (Oprah's Book Club (Paperback))       -0.00686078
13136  The Green Mile: Coffey's Hands (Green Mile Ser...        -0.0069872
13137  A Beautiful Mind: The Life of Mathematical Gen...       -0.00700516
13138                                          4 Blondes       -0.00705296
13139                Primary Colors: A Novel of Politics       -0.00710654
13140                                          Contagion       -0.00719444
13141                                         Dead Sleep       -0.00731026
1

In [11]:
complete.loc[["User_8", "User_67544"]].dropna(thresh=2, axis=1)

Book-Title  Clara Callan
User-ID                 
User_8               5.0
User_67544           8.0

In [12]:
from spotlight.interactions import Interactions
from spotlight.evaluation import rmse_score

#### This doesn't work as expected

In [13]:
user_itempreds = predict_user_ratings(i1, complete)
test = pd.DataFrame(np.vstack((user_itempreds, i1, complete.columns))).T.dropna(thresh=3, axis=0)
test.sort_values(1, ascending=False).head(10)

0  1                       2
2039   5.10612  5            Clara Callan
10488  1.42583  0  The Kitchen God's Wife

In [14]:
from sklearn.decomposition import NMF, TruncatedSVD, dict_learning
import time

In [174]:
def decompose_matrix(matrix, decomposer, evaluate=False):
    """
        matrix: 2D numpy array or pandas dataframe
        decomposer: sklearn class that performs the decomposition.
                    The decomposer can be any object but it is expected
                    to implement an API similar to sklearn with a
                    `fit_transform` method and a `components_` attribute.
        evaluate: bool, whether to also evaluate the model.
    
        Decomposes a matrix with the given method and returns the two
        decomposed vectors. If evaluate=True then this also returns an
        evaluation of the decomposition.
        
        Since we don't want to treat missing values as zeros for evaluation
        but we need numbers to perform the decomposition. One way to test
        this is to add one to all the existent values, then replace NaNs
        with zeros, perform the matrix decomposition, reconstruct the matrix,
        and finally subtract one and calculate the metrics. For the resulting
        decomposition remember to subtract 1 from every future reconstruction,
        and also do min-max scaling (since recostruction may have negative or
        too-high values).        
    """
    
    matrix = (matrix+int(evaluate)).fillna(0).values

    # perform the decomposition
    User = decomposer.fit_transform(matrix)
    Item = decomposer.components_
    
    if evaluate:
        # Reconstruct the matrix and replace negatives with NaNs & clip values
        reconstructed = User @ Item - 1
        reconstructed[reconstructed<0] = np.nan
        reconstructed[reconstructed>10] = 10
    
        # R_squared, pearson correlation, and cosine similarity between 
        # the reconstructed and original matrices
        r2 = compute_evaluation_metric(matrix, reconstructed, "R2")
        rmse = compute_evaluation_metric(matrix, reconstructed, "RMSE")
        corr = similirity_evaluation(reconstructed.ravel(), matrix.ravel())[0][0]
        cosine = similirity_evaluation(reconstructed.ravel(), matrix.ravel(), "cosine")[0][0]
    
        print("RSq: {}\nRMSE: {}\nPearson: {}\nCosine similarity: {}".format(r2, rmse, corr, cosine))
        return User, Item
    
    return User, Item



In [175]:
decomposer = TruncatedSVD(100)
U, I = decompose_matrix(complete, decomposer, evaluate=True)

/home/kmourat/venv/lib/python3.6/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in greater


RSq: 0.8009940018759327
RMSE: 3.916204465187512
Pearson: 0.3103366760774409
Cosine similarity: 0.4744121995336806


In [16]:
# find rows with common ratings with i2
ff3 = complete.T[((~np.isnan(i2)) & (~np.isnan(complete))).any(0)].T
ff3

Book-Title   A Patchwork Planet (Ballantine Reader's Circle)  A Time to Kill  \
User-ID                                                                        
User_100004                                              NaN             NaN   
User_100009                                              NaN             NaN   
User_10001                                               NaN             NaN   
User_100010                                              NaN             NaN   
User_100038                                              NaN             NaN   
User_100053                                              NaN             NaN   
User_100066                                              NaN             NaN   
User_100088                                              NaN             0.0   
User_100098                                              NaN             NaN   
User_100115                                              NaN             NaN   
User_100119                                              NaN             NaN   
User_100188                                              NaN             NaN   
User_100223                                              NaN             NaN   
User_100227                                              NaN             NaN   
User_100251                                              NaN             NaN   
User_100264                                              NaN             NaN   
User_10030                                               NaN             0.0   
User_100361                                              NaN             NaN   
User_100393                                              NaN             NaN   
User_100410                                              NaN             NaN   
User_100454                                              NaN             NaN   
User_100459                                              NaN            10.0   
User_100460                                              NaN             NaN   
User_100506                                              NaN             NaN   
User_100517                                              NaN             NaN   
User_100519                                              NaN             NaN   
User_100546                                              NaN             NaN   
User_100578                                              NaN             NaN   
User_100590                                              NaN             NaN   
User_100594                                              NaN             NaN   
...                                                      ...             ...   
User_99242                                               NaN             NaN   
User_99252                                               NaN             NaN   
User_99268                                               NaN             NaN   
User_99277                                               NaN             NaN   
User_99281                                               NaN             NaN   
User_99312                                               NaN             NaN   
User_99313                                               NaN             NaN   
User_99314                                               NaN             NaN   
User_99328                                               NaN             NaN   
User_99347                                               NaN             NaN   
User_99386                                               NaN             NaN   
User_99390                                               NaN             NaN   
User_99400                                               NaN             NaN   
User_99434                                               NaN             NaN   
User_99441                                               NaN             NaN   
User_99469                                               NaN             NaN   
User_99523                                               NaN             NaN   


In [17]:
ff3.shape

(12490, 38)

In [18]:
# drop rows and columns with less than 5 common ratings
nnans3 = ff3.dropna(thresh=2, axis=1).dropna(thresh=2, axis=0)
nnans3

Book-Title   A Patchwork Planet (Ballantine Reader's Circle)  A Time to Kill  \
User-ID                                                                        
User_10030                                               NaN             0.0   
User_100459                                              NaN            10.0   
User_100846                                              NaN             NaN   
User_101209                                              NaN             NaN   
User_101305                                              NaN             NaN   
User_101876                                              NaN             NaN   
User_102275                                              NaN             NaN   
User_102647                                              NaN            10.0   
User_102967                                              NaN             0.0   
User_10314                                               NaN             NaN   
User_103336                                              0.0             NaN   
User_104165                                              NaN             NaN   
User_104429                                              NaN            10.0   
User_10447                                               NaN             NaN   
User_104636                                              NaN            10.0   
User_104665                                              NaN             8.0   
User_105028                                              NaN             8.0   
User_105374                                              NaN             NaN   
User_105517                                              NaN             0.0   
User_10560                                               NaN             NaN   
User_105937                                              NaN             NaN   
User_105979                                              NaN             0.0   
User_106225                                              NaN             0.0   
User_106331                                              NaN             NaN   
User_106816                                              NaN             NaN   
User_107301                                              NaN             NaN   
User_107324                                              NaN             NaN   
User_108005                                              NaN             NaN   
User_108285                                              NaN             0.0   
User_108485                                              NaN             NaN   
...                                                      ...             ...   
User_93085                                               NaN             NaN   
User_93092                                               NaN             NaN   
User_93363                                               NaN             NaN   
User_93426                                               NaN             NaN   
User_93629                                               NaN             NaN   
User_93631                                               NaN             NaN   
User_94242                                               NaN             NaN   
User_94347                                               NaN             NaN   
User_94396                                               NaN            10.0   
User_95173                                               NaN             NaN   
User_95353                                               NaN             0.0   
User_95359                                               NaN             NaN   
User_95393                                               NaN             NaN   
User_95903                                               NaN             NaN   
User_95921                                               NaN             NaN   
User_95932                                               NaN             0.0   
User_96054                                               NaN             0.0   


In [19]:
nnans3.shape

(1065, 38)

In [20]:
# Find the similarity between the new vector and the candidates
i2_short = nnans3.loc[:,"A Patchwork Planet (Ballantine Reader's Circle)"]
similirity_evaluation(i2_short, nnans3.T, "cosine").round(2)

array([[1.  , 0.  , 0.05, 0.21, 0.04, 0.  , 0.  , 0.25, 0.18, 0.11, 0.16,
        0.08, 0.08, 0.08, 0.03, 0.13, 0.06, 0.31, 0.  , 0.  , 0.05, 0.41,
        0.1 , 0.  , 0.04, 0.  , 0.  , 0.06, 0.08, 0.05, 0.11, 0.06, 0.09,
        0.05, 0.12, 0.05, 0.05, 0.12]])

In [21]:
similirity_evaluation(i2_short, nnans3.loc[:,"An Italian Affair"].T,
                      "cosine").round(2)

array([[0.21]])

In [22]:
nnans3.shape, i2_short.shape

((1065, 38), (1065,))

In [23]:
# check original (only common values) to see if 0.44 correlation appears plausible 
complete.loc[:,["An Italian Affair",
                "A Patchwork Planet (Ballantine Reader's Circle)"]].dropna(thresh=2, axis=0)

Book-Title   An Italian Affair  \
User-ID                          
User_11400                 8.0   
User_194600                7.0   

Book-Title   A Patchwork Planet (Ballantine Reader's Circle)  
User-ID                                                       
User_11400                                               7.0  
User_194600                                              6.0

## Experimenting with tensorflow for decomposition

This didn't seem to work very well.

In [24]:
from tqdm import tqdm

In [25]:
import tensorflow as tf

In [26]:
sess = tf.Session()

In [27]:
mask = (complete * 0 + 1).fillna(0).values
mask

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
complete.shape

(12490, 13150)

In [38]:
n = 50

InputTensor = tf.placeholder(tf.float32, shape=(None, complete.shape[1]))

UserTensor = tf.Variable(tf.random_normal((complete.shape[0],n), dtype=tf.float32),
                         dtype=tf.float32)
ItemTensor = tf.Variable(tf.random_normal((n, complete.shape[1]), dtype=tf.float32),
                              dtype=tf.float32)

# a mask of missing values (true when present, false when nan)
Mask = tf.constant(mask.astype(np.float32))
reg_lambda = tf.constant(0.5)

norms = tf.add(tf.sqrt(tf.reduce_sum(tf.square(UserTensor))),
              tf.sqrt(tf.reduce_sum(tf.square(ItemTensor))))

# Using the mask, we force the loss to be computed only on known values
mul = tf.multiply(Mask, tf.matmul(UserTensor, ItemTensor,))
diffs = tf.subtract(InputTensor, mul)
squares = tf.square(diffs)
loss = tf.add(tf.reduce_sum(squares), tf.multiply(reg_lambda, norms))

opt = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)
sess.run(tf.global_variables_initializer())


In [39]:
for i in range(200):
    o, l = sess.run([opt, loss], feed_dict={InputTensor: complete.fillna(0).astype(np.float32).values})
    print(f"Epoch: {i}, Loss: {l:.2f}")

Epoch: 0, Loss: 28692564.00
Epoch: 1, Loss: 19756440.00
Epoch: 2, Loss: 13771133.00
Epoch: 3, Loss: 9887452.00
Epoch: 4, Loss: 7423917.50
Epoch: 5, Loss: 5876934.00
Epoch: 6, Loss: 4894670.00
Epoch: 7, Loss: 4243662.50
Epoch: 8, Loss: 3777818.75
Epoch: 9, Loss: 3412460.25
Epoch: 10, Loss: 3103042.00
Epoch: 11, Loss: 2828696.25
Epoch: 12, Loss: 2580980.75
Epoch: 13, Loss: 2357169.25
Epoch: 14, Loss: 2156456.50
Epoch: 15, Loss: 1977980.00
Epoch: 16, Loss: 1820148.38
Epoch: 17, Loss: 1680752.12
Epoch: 18, Loss: 1557328.12
Epoch: 19, Loss: 1447465.12
Epoch: 20, Loss: 1349021.88
Epoch: 21, Loss: 1260267.75
Epoch: 22, Loss: 1179893.62
Epoch: 23, Loss: 1106914.25
Epoch: 24, Loss: 1040566.75
Epoch: 25, Loss: 980211.19
Epoch: 26, Loss: 925260.31
Epoch: 27, Loss: 875173.12
Epoch: 28, Loss: 829446.44
Epoch: 29, Loss: 787622.88
Epoch: 30, Loss: 749296.81
Epoch: 31, Loss: 714084.06
Epoch: 32, Loss: 681610.88
Epoch: 33, Loss: 651505.44
Epoch: 34, Loss: 623425.81
Epoch: 35, Loss: 597124.62
Epoch: 36,

In [40]:
U = sess.run(UserTensor)
I = sess.run(ItemTensor)

In [41]:
(U @ I).max(), (U @ I).min()

(93.80846, -98.86373)

In [42]:
reconstructed = U @ I

r2 = compute_evaluation_metric(complete, reconstructed, "R2")
rmse = compute_evaluation_metric(complete, reconstructed, "RMSE")
corr = similirity_evaluation(reconstructed.ravel(), complete.values.ravel())[0][0]
cosine = similirity_evaluation(reconstructed.ravel(), complete.values.ravel(), "cosine")[0][0]

print("RSq: {}\nRMSE: {}\nPearson: {}\nCosine similarity: {}".format(r2, rmse, corr, cosine))

RSq: 0.9923013192470189
RMSE: 0.33378412402625257
Pearson: 0.020808072746854676
Cosine similarity: 0.02528973086618768


## This seems to be working nicely

In [43]:
reconstructed = reconstructed.clip(0,10)

r2 = compute_evaluation_metric(complete*mask, reconstructed*mask, "R2")
rmse = compute_evaluation_metric(complete*mask, reconstructed*mask, "RMSE")
corr = similirity_evaluation(reconstructed.ravel()*mask.ravel(),
                             complete.values.ravel()*mask.ravel())[0][0]
cosine = similirity_evaluation(reconstructed.ravel()*mask.ravel(),
                               complete.values.ravel()*mask.ravel(), "cosine")[0][0]

print("RSq: {}\nRMSE: {}\nPearson: {}\nCosine similarity: {}".format(r2, rmse, corr, cosine))

RSq: 0.9938639148548395
RMSE: 0.29799104246490116
Pearson: 0.8159394505510064
Cosine similarity: 0.9979132318557788


In [47]:
xx = 9
preds = pd.DataFrame([complete.iloc[xx].values, reconstructed[xx], complete.columns], 
             index=["User", "Preds", "Items"]).T.sort_values("User", ascending=False)
preds

User        Preds                                              Items
5624    10      9.98883  Living the Simple Life: A Guide to Scaling Dow...
10050   10           10                 The Five People You Meet in Heaven
12853    9      8.99655  Who Moved My Cheese? An Amazing Way to Deal wi...
2830     9      8.98362   Divine Secrets of the Ya-Ya Sisterhood : A Novel
7520     8      7.99427         RAMONA FOREVER (Ramona Quimby (Paperback))
10617    6      6.01415                             The Lilac Bus: Stories
9107     6      5.97508                                        Term Limits
4486     6      6.00489                               Honourable Schoolboy
5143     0    0.0591984                                       Kaleidoscope
10569    0            0                   The Last Time They Met : A Novel
12487    0  1.87584e-05                                           Vanished
12306    0   0.00512643  Tuesdays with Morrie: An Old Man, a Young Man,...
12197    0   0.00284042              Total Recall: A V.I. Warshawski Novel
11836    0    0.0094772                                        The Wedding
11208    0            0                                 The Return Journey
11160    0   0.00380116                                          The Ranch
10684    0    0.0178558                          The Lovely Bones: A Novel
10653    0    0.0234448                                 The Long Road Home
3082     0   0.00492968                                             Echoes
10212    0            0                            The Greatest Generation
4915     0  0.000391535                                         Irish Born
4172     0    0.0114245                            Happy Are the Oppressed
9804     0            0                                   The Devil's Code
9440     0            0                                The Bourne Identity
8171     0    0.0048483            Seven Habits Of Highly Effective People
8059     0            0                                         Seabiscuit
4801     0            0                                  In Silence (Mira)
7491     0            0                                    Q Is for Quarry
7310     0            0    Politika (Tom Clancy's Power Plays (Paperback))
7156     0            0                               Pearl Cove (Donovan)
...    ...          ...                                                ...
13120  NaN      3.61313  Your Oasis on Flame Lake (Ballantine Reader's ...
13121  NaN            0  Your Pregnancy: Week by Week (Your Pregnancy S...
13122  NaN            0  Your Sixth Sense: Unlocking the Power of Your ...
13123  NaN            0                                    Youth in Revolt
13124  NaN            0                                          Yukon Ho!
13125  NaN           10                    Yuletide Brides (2 Novels in 1)
13126  NaN            0                                    Z for Zachariah
13127  NaN            0                                       ZEKE AND NED
13128  NaN      6.13014                                      Zeit im Wind.
13129  NaN      7.18933                                       Zeke and Ned
13130  NaN            0                                       Zen Attitude
13131  NaN           10          Zen and the Art of Motorcycle Maintenance
13132  NaN            0  Zen and the Art of Motorcycle Maintenance: An ...
13133  NaN      4.32921  Zen and the Art of Murder (St. Martin's Minota...
13134  NaN            0                          Zen in the Art of Writing
13135  NaN            0                                   Zero at the Bone
13136  NaN            0                                                Zia
13137  NaN      4.49645          Zlata's Diary: A Child's Life in Sarajevo
13138  NaN      1.23089           Zlata's Diary: A Childs Life in Sarajevo
13139  NaN            0                                             Zodiac
13140  NaN       4.7188                           Zodiac: The Eco-Thriller
13141  NaN            0  Zoey P

In [53]:
preds[preds["User"].isna()].sort_values("Preds", ascending=False)

User Preds                                              Items
6470   NaN    10                               My Little Blue Dress
5119   NaN    10         Just Me and My Dad (Golden Look-Look Book)
11304  NaN    10                                           The Scar
12397  NaN    10                                     Under the Lake
1449   NaN    10                                          Bluebeard
3548   NaN    10                                       First Avenue
5133   NaN    10                                   Justice for Some
10417  NaN    10                                      The Inheritor
7216   NaN    10  Pessimisms: Famous (and not so famous) Observa...
9282   NaN    10                         The Ballad of Typhoid Mary
12411  NaN    10                                          Undressed
7226   NaN    10                Peter the Great: His Life and World
7231   NaN    10                               Phantom of the Opera
1456   NaN    10                                         Body Check
9278   NaN    10                                    The Bad Witness
9277   NaN    10                                      The Bad Place
7237   NaN    10            Phule's Company (Phule's Company, No 1)
7238   NaN    10                 Phule's Paradise (Phule's Company)
12423  NaN    10                                        Unholy Fire
12424  NaN    10       Unicorn Point (Apprentice Adept (Paperback))
11289  NaN    10  The Salmon of Doubt: Hitchhiking the Galaxy On...
11288  NaN    10                                The Salmon of Doubt
1405   NaN    10                                         Blood Work
1403   NaN    10                                        Blood Trail
9290   NaN    10                                          The Beach
7198   NaN    10                                   Perfect Stranger
12358  NaN    10                                     U.S.S. Seawolf
1502   NaN    10  Born in a Small Town (Harlequin SuperRomance 936)
1527   NaN    10                                             Brazil
3500   NaN    10  Final Jeopardy (Alexandra Cooper Mysteries (Pa...
...    ...   ...                                                ...
5750   NaN     0                                       Love in Vein
5751   NaN     0                        Love in the Time of Cholera
5752   NaN     0  Love in the Time of Cholera (Penguin Great Boo...
5753   NaN     0                              Love to Love You Baby
5755   NaN     0                                      Love's Encore
5718   NaN     0                                     Louisa Elliott
5757   NaN     0  Love's Legacy (Leisure Super Special January 1...
5760   NaN     0                            Love, Greg &amp; Lauren
5761   NaN     0  Love, Medicine and Miracles : Lessons Learned ...
5762   NaN     0                               Love: A User's Guide
5764   NaN     0                                Lovely in Her Bones
5765   NaN     0                                       Lover Beware
5746   NaN     0                                       Love and War
5745   NaN     0                                  Love and Marriage
5744   NaN     0                                  Love a Dark Rider
5742   NaN     0  Love With the Proper Husband (Avon Historical ...
5741   NaN     0                        Love With a Scandalous Lord
5739   NaN     0        Love Takes Wing (Love Comes Softly, Book 7)
5734   NaN     0                                      Love Medicine
5733   NaN     0       Love Me Forever (Sherring Cross (Paperback))
5732   NaN     0         Love Lies Bleeding: A China Bayles Mystery
5730   NaN     0  Love Is Hell: Special New Mini-Jumbo Edition W...
5729   NaN     0                                    Love Invents Us
5727   NaN     0                        Love Inspired: Finding Hope
5725   NaN     0      Love Comes Softly (Love Comes Softly, Book 1)
5724   NaN     0                                     Love By Design
5721   NaN     0      Love Always: A Novel (Vintage Contempora

In [58]:
def get_reconstructed_ratings(user, Users, Items, df):
    
    user = np.atleast_2d(user)
    
    vec = user @ Items
    
    preds = pd.DataFrame([complete.iloc[xx].values, reconstructed[xx], complete.columns], 
                         index=["User", "Preds", "Items"]).T
    return preds[preds["User"].isna()].sort_values("Preds", ascending=False)

In [59]:
get_reconstructed_ratings(i1, U, I, complete)

ValueError: shapes (1,13150) and (50,13150) not aligned: 13150 (dim 1) != 50 (dim 0)

In [61]:
U.shape, I.shape

((12490, 50), (50, 13150))

In [125]:
xxx = complete.iloc[0].fillna(0).values.reshape((1,-1))
((I @ xxx.T).T @ I).shape

(1, 13150)

In [126]:
xxx.shape, I.shape, U.shape

((1, 13150), (50, 13150), (12490, 50))

In [131]:
(xxx @ I.T) @ U.T

array([[ 464.14839094,  334.77402609, -237.11318784, ...,  -90.62886511,
          68.11662654,   73.31544791]])

In [132]:
U @ (I @ xxx.T)

array([[ 464.14839094],
       [ 334.77402609],
       [-237.11318784],
       ...,
       [ -90.62886511],
       [  68.11662654],
       [  73.31544791]])

In [139]:
((xxx @ I.T)).shape

(1, 50)

In [134]:
(U @ I).shape

(12490, 13150)

In [138]:
(xxx @ (U@I)).shape

ValueError: shapes (1,13150) and (12490,13150) not aligned: 13150 (dim 1) != 12490 (dim 0)

In [140]:
tsvd.components_.shape

(10, 13150)

In [141]:
U.shape

(12490, 50)

In [143]:
xxx.shape

(1, 13150)

In [156]:
similirity_evaluation(xxx @ I.T, U).shape

(1, 12490)

In [151]:
similirity_evaluation(xxx @ I.T, U).max()

0.5084103157007126

In [152]:
similirity_evaluation(xxx @ I.T, U).min()

-0.463650119000801

In [153]:
np.percentile(similirity_evaluation(xxx @ I.T, U), 90)

0.20640708205877015

In [155]:
(similirity_evaluation(xxx @ I.T, U)<0.25).mean()

0.944275420336269

In [117]:
complete.iloc[0].fillna(0).values

array([0., 0., 0., ..., 0., 0., 0.])

In [119]:
tsvd.transform(complete.iloc[0].fillna(0).values.reshape((1,-1)))

array([[ 1.90254889,  1.95334065, -3.24689092,  2.88255786, -6.49299172,
        -1.20928313,  0.58378095,  1.14808991, -0.04961669, -0.21718322]])

## Experimenting with apriori and association rules

This seems to work great.

## Nearest Neighbor method (might be interesting with better vector representations)